<a href="https://colab.research.google.com/github/VayuSarangam/cmbs-cashflow/blob/main/notebooks/02_structured_%20credit_waterfall_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Start Colab correctly, clone with existing repo.

In [1]:
%cd /content
!rm -rf cmbs-cashflow
!git clone https://github.com/VayuSarangam/cmbs-cashflow.git
%cd /content/cmbs-cashflow
!pip -q install -r requirements.txt
!git status


/content
Cloning into 'cmbs-cashflow'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 98 (delta 45), reused 23 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 885.15 KiB | 8.51 MiB/s, done.
Resolving deltas: 100% (45/45), done.
/content/cmbs-cashflow
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


Make sure collateral cashflows exist, waterfall needs outputs/collateral_cashflows.csv

In [2]:
!ls -lah outputs || true
!ls -lah artifacts || true


total 12K
drwxr-xr-x 2 root root 4.0K Feb 16 08:14 .
drwxr-xr-x 8 root root 4.0K Feb 16 08:14 ..
-rw-r--r-- 1 root root    1 Feb 16 08:14 .gitkeep
total 12K
drwxr-xr-x 3 root root 4.0K Feb 16 08:14 .
drwxr-xr-x 8 root root 4.0K Feb 16 08:14 ..
drwxr-xr-x 2 root root 4.0K Feb 16 08:14 01_outputs_v1


copy outputs/collateral_cashflows.csv from artifacts

In [3]:
!mkdir -p outputs
!cp -f artifacts/01_outputs_v1/collateral_cashflows.csv outputs/collateral_cashflows.csv
!ls -lah outputs


total 68K
drwxr-xr-x 2 root root 4.0K Feb 16 08:14 .
drwxr-xr-x 8 root root 4.0K Feb 16 08:14 ..
-rw-r--r-- 1 root root  56K Feb 16 08:14 collateral_cashflows.csv
-rw-r--r-- 1 root root    1 Feb 16 08:14 .gitkeep


Commit checkpoint

In [4]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


Create the 3 required input config CSVs, Put these in data/ so the repo is self-contained:

##  data/tranche_structure.csv

In [5]:
%%writefile data/tranche_structure.csv
DealID,Tranche,ClassType,OrigBalance,CouponType,CouponRate,SpreadBps,DayCount,PayFreq,InterestPriority,PrincipalPriority,LossPriority
DEAL_001,A,Senior,300000000,FLOAT,0.0000,175,ACT/360,MONTHLY,1,1,6
DEAL_001,B,Mezz,100000000,FLOAT,0.0000,325,ACT/360,MONTHLY,2,2,5
DEAL_001,XS,IO,0,IO_STRIP,0.0100,0,ACT/360,MONTHLY,0,0,0
DEAL_001,RR,Residual,0,RESIDUAL,0.0000,0,ACT/360,MONTHLY,99,99,1


Overwriting data/tranche_structure.csv


Meaning (tight):

OrigBalance: starting note balance for principal-paying tranches (A/B). IO/Residual can be 0.

InterestPriority: lower number paid first (A before B).

PrincipalPriority: sequential = 1 for A, 2 for B.

LossPriority: reverse sequential (Residual first, then lowest tranche, then senior last).

## data/deal_terms_fees_triggers.csv

In [6]:
%%writefile data/deal_terms_fees_triggers.csv
DealID,ServicerFeeBps,TrusteeFeeMonthly,AdminFeeMonthly,OCTriggerOn,OCThreshold,ICTriggerOn,ICThreshold
DEAL_001,10,15000,5000,N,0.00,N,0.00


Overwriting data/deal_terms_fees_triggers.csv


Implementing fees now.

## data/waterfall_rules.csv

In [7]:
%%writefile data/waterfall_rules.csv
DealID,PrincipalRule,LossRule,InterestShortfallRule
DEAL_001,SEQUENTIAL,REVERSE_SEQUENTIAL,CARRY_FORWARD


Overwriting data/waterfall_rules.csv


In [8]:
%cd /content
!rm -rf cmbs-cashflow
!git clone https://github.com/VayuSarangam/cmbs-cashflow.git
%cd /content/cmbs-cashflow
!ls -a


/content
Cloning into 'cmbs-cashflow'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 98 (delta 45), reused 23 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 885.15 KiB | 8.35 MiB/s, done.
Resolving deltas: 100% (45/45), done.
/content/cmbs-cashflow
.   artifacts  .git	   LICENSE    outputs	 requirements.txt
..  data       .gitignore  notebooks  README.md  src


In [9]:
!git config --global user.email "vayu.sarangam01@gmail.com"
!git config --global user.name "VayuSarangam"


Commit checkpoint

In [10]:
!git add data/tranche_structure.csv data/deal_terms_fees_triggers.csv data/waterfall_rules.csv
!git commit -m "Add Notebook 03 waterfall input configs (tranches, fees, rules)"
!git push


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address


In [11]:
%cd /content/cmbs-cashflow
!git status


/content/cmbs-cashflow
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
